In [1]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pytz
import functions_framework

In [2]:
def get_articles():
    with open('./get_headers_news_local.json', 'r') as json_file:
        websites = json.load(json_file)

    # サイト毎に見出しを見に行き、前回との差分を見る
    for site in websites:
        response =requests.get(site["url"])
        soup = BeautifulSoup(response.content, 'html.parser')
        headlines = soup.select(site["selector"])

        # 前回の見出しを保存しておく
        site["get_title_before"] = site["get_title"]
        # 取得した見出しを保存
        site["get_title"] = headlines[0].text.strip()

        # 結果を表示
        print(
            site["head"] +
            "\nNew title\n" +
            str(site["get_title"]) + 
            "\nBefore title\n" +
            str(site["get_title_before"])
        )

        # 差分のが出たかを見て、値として保存
        if site["get_title_before"] != site["get_title"]:
            print("Update\n")
            site["update"] = True
        else:
            print("No update\n")        
            site["update"] = False

    # 処理の結果を保存
    with open("./get_headers_news_local.json", "w", encoding="utf-8") as json_file:
        json.dump(websites, json_file, ensure_ascii=False, indent=4)
        
    # スクレイピングの結果から、アップデートがあったものだけ配列に追加
    updates = []
    for web in websites:
        if web["update"] == True:
            updates.append(web)

    # 一つ以上アップデートがある場合、ファイルを上書きして保存
    if 0 < len(updates):
        print("get updates")
        japan_tz = pytz.timezone('Asia/Tokyo')
        current_time_japan = datetime.now(japan_tz)
        formatted_time = current_time_japan.strftime('%Y/%m/%d %H:%M')

        new_object = {
            "date": formatted_time,
            "updates": updates
        }

        # 現状を読み込んで変数に格納
        with open ("update.json", "r") as update:
            update_json = json.load(update)
            update_json.append(new_object)

        # 更新分のオブジェクトを配列を追加して、jsonファイルとして保存
        with open ("get_headers_news_local_update.json", "w") as update:
            json.dump(update_json, update, ensure_ascii=False, indent=4)

        # フロントエンド用にも保存する
        with open ("../docs/data/get_headers_news_local_update.json", "w") as update:
            json.dump(update_json, update, ensure_ascii=False, indent=4)
        print(update_json[len(update_json)-1])
    else:
        print("no update")

In [3]:
get_articles()

日経電子版 速報
New title
東証10時　下げ幅拡大　一時200円超安　円一段高で
Before title
育児時短勤務、賃金の1割給付を検討　2歳未満対象
Update

NHKニュース 速報・新着一覧
New title
【随時更新】ガザで拘束12人解放 収監のパレスチナ人30人釈放
Before title
中国 李強首相 半導体などの輸出規制強めるアメリカけん制
Update

Yahoo!ニュース 速報
New title
【16日ぶりの救出】インドで建設中のトンネルが崩落、作業員41人が閉じ込められていた事故　全員が救出「健康状態は良好」
Before title
戦闘休止を2日間延長へ　国連機関の責任者「必要なのは長期の休戦」
Update

get updates
{'date': '2023/11/29 10:14', 'updates': [{'head': '日経電子版 速報', 'media': 'NIKKEI', 'url': 'https://www.nikkei.com/news/category/', 'selector': '#CONTENTS_MAIN > div:nth-child(1) > h3 > a > span', 'get_title': '東証10時\u3000下げ幅拡大\u3000一時200円超安\u3000円一段高で', 'get_title_before': '育児時短勤務、賃金の1割給付を検討\u30002歳未満対象', 'update': True}, {'head': 'NHKニュース 速報・新着一覧', 'media': 'NHK', 'url': 'https://www3.nhk.or.jp/news/catnew.html', 'selector': '#main > article > section > div > ul > li:nth-child(1) > dl > dd > a > em', 'get_title': '【随時更新】ガザで拘束12人解放 収監のパレスチナ人30人釈放', 'get_title_before': '中国 李強首相 半導体などの輸出規制強めるアメリカけん制', 'update': True}, {'head': 'Yahoo!ニュース 速報', 'm